In [1]:
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np 
import pandas as pd 
import io
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train = x_train.reshape(60000, num_features)
x_test = x_test.reshape(10000, num_features)


# Normalize images value from [0, 255] to [0, 1].
x_train = x_train / 255
x_test = x_test /255

x_train

11501568/11490434 [==============================] - 0s 0us/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [3]:
# MNIST dataset parameters.

num_classes = 10 # 0 to 9 digits

num_features = 784 # 28*28

# Training parameters.

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

In [4]:
# Use tf.data API to shuffle and batch data.
num_batches = int(x_train.shape[0] / batch_size)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(np.random.randn(784, 10).astype(np.float32))


# Bias of shape [10], the total number of classes.
B = tf.Variable(np.random.randn(10).astype(np.float32))

In [6]:
# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.add(tf.matmul(x, W), B))
    

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth = num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)    
    # Compute cross-entropy.
    loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return loss

In [7]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# Optimization process. 

optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, B])
    # Stochastic gradient descent optimizer.
    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, B]))

In [9]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)
    if step % display_step == 0:

        #Obtain Predictions
        pred = logistic_regression(batch_x)
        #Ccompute loss
        loss = cross_entropy(pred, batch_y)
        #Compute Accuracy
        acc = accuracy(pred, batch_y)
        #print accuracy
        print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 50, loss: 265.7037353515625, accuracy: 0.81640625
step: 100, loss: 136.5028076171875, accuracy: 0.87109375
step: 150, loss: 129.7585906982422, accuracy: 0.8984375
step: 200, loss: 138.41461181640625, accuracy: 0.875
step: 250, loss: 82.82695007324219, accuracy: 0.921875
step: 300, loss: 90.63404083251953, accuracy: 0.9140625
step: 350, loss: 95.4364242553711, accuracy: 0.921875
step: 400, loss: 85.93441772460938, accuracy: 0.90625
step: 450, loss: 71.66111755371094, accuracy: 0.92578125
step: 500, loss: 138.20355224609375, accuracy: 0.890625
step: 550, loss: 121.72308349609375, accuracy: 0.8984375
step: 600, loss: 134.63497924804688, accuracy: 0.9140625
step: 650, loss: 126.19697570800781, accuracy: 0.8984375
step: 700, loss: 55.838897705078125, accuracy: 0.93359375
step: 750, loss: 110.46906280517578, accuracy: 0.8984375
step: 800, loss: 129.39427185058594, accuracy: 0.88671875
step: 850, loss: 71.134765625, accuracy: 0.94140625
step: 900, loss: 115.42486572265625, accuracy: 0.9

In [10]:
# Test model on validation set.
pred = logistic_regression(x_test)
a = accuracy(pred, y_test)
print(f"Test Accuracy: {a}")

Test Accuracy: 0.8550000190734863
